# GMAG Local DASP Tutorial

This note books is the same as the Colab notebook but assumes you have GMAG installed locally.

Installation instructions can be found [here](https://github.com/kylermurphy/DASP_2024/tree/main/GMAG#gmag-installation).


Check if we can load GMAG

In [ ]:
import gmag.config as config

config_path = config.get_config_file()
print(f'Path to the gmag config file: {config_path}')


## Start of the DASP GMAG Tutorial

# GMAG

```gmag``` is an open-source Python library which provides rapid access to ground-based magnetometer data from several array in a common data format.

- [GMAG paper](https://www.frontiersin.org/articles/10.3389/fspas.2022.1005061/full)
- [GMAG Website/Docs](https://kylermurphy.github.io/gmag/)
- [GMAG repository](https://github.com/kylermurphy/gmag)

Why magnetometers?
- Their a unique data set in Heliophysics
 - One of the few [globally distributed data sets](https://kylermurphy.github.io/gmag/stations)
 - One of the few [long term data sets](https://doi.org/10.1016/j.jastp.2011.02.018) - span multiple solar cycles
- They can probe various ionosphere and magnetosphere phenomena
 - [Radiation belts](https://doi.org/10.1038/nphys3799)
 - [Aurora](https://doi.org/10.1002/2013JA018889)
 - [Substorms](https://doi.org/10.1002/2013JA018979)
 - [ULF waves](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1029/2005JA011007)

Why was ```gmag``` developed?
- I wanted to take advantage of global magnetometer measurements for both case studies and statistical studies.
  - required code to read data from multiple providers into a common data format
- I wanted access to cool Python code.
- I wanted to move away from IDL (expensive, falling behind or at the very least trailing very far).

Why ```gmag``` when other resources exist?
- GMAG downloads data from the source which can provide **upto date** and **higher cadence** data
- GMAG is **light weight**
- GMAG uses **Pandas DataFrames which work well within the Python framework** and with analysis (numpy, scipy), plotting (matplotlib, seaborn), and machine learnging (sck-kit learn) packages.

## Loading Data

```gmag``` is primarily for loading data. Though it will also load station and array meta data, rotate the data into geomagnetic coordinates, and do some very basic cleaning of the data.

**Lets load plot some CARISMA data.**




In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Plot multi-panel plot of the H compoment
# magnetic field for select CARSIMA stations

# import required modules
import gmag.arrays.carisma as carisma

# define start and end date for loading
# assume a single day is loaded
sdate = '2014-11-05 13:25:00'
edate = '2014-11-05 14:25:00'

stn = ['PINA','ISLL','GILL','FCHU','RANK']

# load the data and meta data
car_dat, car_meta=carisma.load(stn,sdate)


**Lets have a look at the data**

In [ ]:
car_dat
car_meta

```gmag``` loads the raw data, rotates it (if required), and also loads the station metadata.

The metadata stores the array and station code, station coordinates, cadence, the data coordinates, PI and Institution.

## Accessing Data

The data and metadata are loaded in a [Pandas DataFrame](https://pandas.pydata.org/docs/index.html).
- [10 minutes to Pandas](https://pandas.pydata.org/docs/user_guide/10min.html#)
- [Pandas DataFrame](https://pandas.pydata.org/docs/user_guide/dsintro.html#dataframe)
- [Get Item](https://pandas.pydata.org/docs/user_guide/10min.html#getitem)
- [Selection by label](https://pandas.pydata.org/docs/user_guide/10min.html#selection-by-label)
- [Selection by position](https://pandas.pydata.org/docs/user_guide/10min.html#selection-by-position)
- [Boolean Indexing](https://pandas.pydata.org/docs/user_guide/10min.html#boolean-indexing) & [masking](https://pandas.pydata.org/docs/user_guide/indexing.html#the-where-method-and-masking)



In [ ]:
# Get Item
# Accessing by label/column name
car_dat['PINA_H']
car_dat[['PINA_H','PINA_D']]

# Accessing by location, still require labels
car_dat.loc['2014-11-05 01:00:00','GILL_D']

# Accessing by position, like an array

car_dat.iloc[:,-3:-1]

# note the index here is a DateTime object
# to make it more like an array reset the index
car_dat.reset_index().iloc[0:10,0:2]

In [ ]:
# Boolean Indexing
# only get indexed values where
car_meta[car_meta['cgm_latitude']>60]


In [ ]:
# Masking
# Only print where indexing is true
lat_mask = (car_meta['latitude'] > 53) & (car_meta['latitude'] < 57)
lat_mask
car_meta[lat_mask]



```gmag``` can load data from [12 arrays](https://kylermurphy.github.io/gmag/arrays) (just over 200 magnetometer stations). Data from these arrays are loaded using the ```load()``` function from the ```array``` subpackage in 4 different modules.

```python
#array sub modules
import gmag.arrays.carisma as carisma
import gmag.arrays.canopus as canopus
import gmag.arrays.image as image
import gmag.arrays.themis as themis
```

All the load functions have the same basic format

```python
load(site: str = ['GILL'],
         sdate='2010-01-01',
         ndays: int = 1,
         edate=None,
         dl=True,
         force=False):    
```

You can access the doc string for each module using ```?``` or ```help(array.module)```


In [ ]:
carisma.load?

### Exercise 1

Try loading CARISMA data from 2018 and 2023.
Do you notice a difference between the two datasets? For simplicity the Gillam station (GILL) on January 1 from each year.

In [ ]:
# load GILL data from 2018-01-01 and 2023-01-01

### Solution

In [ ]:
d1 = carisma.load('GILL',sdate='2018-01-01')
d2 = carisma.load('GILL',sdate='2023-01-01')

In [ ]:
# note the difference here
# because I used
# d1 = carisma.load('GILL',sdate='2018-01-01')
# the data and metadata are returned as a tuple
# the first element is the data, and second the meta data

d1[0].head()
d2[0].head()

## Rotating Data

```gmag``` rotates data from geographic North-East-South (X-Y-Z) to geomagnetic North-East-South (H-D-Z) when it can.

This requires the magnetic declination of the stations. geomagnetic coordinates. These are located in text files seperated by year ```gmag/gmag/Stations/xxxx_station_cgm.txt```.

These text files have been generated for 1990-2019 (inclusively) and haven't been generated for the 2020's yet (working progress).

**Data loaded via the ```themis``` module has undergone extensive processing and is typically returned in local geomagnetic coordinates (HDZ) and so is not rotated when loaded.**




### Utilities

The ```utils``` module provides a number of routines to load station and array information including geographic and geomagnetic coordinates of all the stations.

These are useful for identifying magnetometer stations apart of particular array, or physical location.

In [ ]:
from gmag import utils

#load geographic data for all stations
all_stn = utils.load_station_geo(param='*')
all_stn.head()


In [ ]:
# load geomagnetic data for all stations
# requires a year
all_stn = utils.load_station_coor(param='*',year=2018)
all_stn.tail()

The ```utils``` functions also return a Pandas DataFrame which can be easily masked to identify key station characteristics.

In [ ]:
# print all CARISMA data
car_pos = all_stn['array'] == 'CARISMA'
all_stn[car_pos]

### Exercise 2

Identify all magnetometer stations around geosynchronous orbit, say 6.25 < L < 6.75.

In [ ]:
# put your code here

### Solution

In [ ]:
geo = (all_stn['lshell'] > 6.25) & (all_stn['lshell'] < 6.75)
all_stn[geo]

## Plotting Data

Because ```gmag``` uses Pandas DataFrames plotting is relatively quick and generating nice plots is relatively easy. It even inherently handles time-series plotting quite nicely (formatted time axis).

[Pandas Visualization](https://pandas.pydata.org/docs/user_guide/visualization.html)

[Pandas DataFrame Plotting](https://pandas.pydata.org/docs/reference/frame.html#plotting)

A simple plot

In [ ]:
# import required modules
import gmag.arrays.carisma as carisma
import numpy as np
import matplotlib.pyplot as plt

# define start and end date for plotting
# use start date for loading data
sdate = '2014-11-05 13:25:00'
edate = '2014-11-05 14:25:00'

# load data
car_dat, car_meta=carisma.load(['PINA','ISLL','GILL','FCHU','RANK'],sdate)



In [ ]:
car_dat['GILL_H'].plot()
#car_dat.plot(y=['GILL_H','FCHU_H','ISLL_H','PINA_H'])
#car_dat.plot(y=['GILL_H','FCHU_H','ISLL_H','PINA_H'],subplots=True,ylabel='nT')

A more complex plot

In [ ]:
# plot a stacked plot of CARISMA the H component
# magnetic field for stations along the Churchill line

# define component for plotting
comp='H'

# find the columns from the loaded DataFrame that have comp
# in the title, these are the columns that will be plotted
p_col = [col for col in car_dat.columns if col[-1] == comp]

# determine the shift to apply to each time series so that they don't
# overlatp

# the shift is determined using the DataFrame returned by the describe()
# method which stores the DataFrame stats including max and min of each column
# only use columns from p_col and values between the start and end of plotting
# defined by sdate and edate
# the shift in the y direction is defined by 1.5 times the range of the series
y_shift = np.array([(val['max']-val['min'])/1.5 for col_h, val in car_dat[sdate:edate][p_col].describe().items()])

# the cumsum() method determines the cumalitative sum up
# to each index
# the cumsum() ensures timeseries don't overlap
y_shift = (y_shift-y_shift.min()).cumsum()

# plot p_col columns of the data frame between sdate and edate
# subtract the mean from each time series and apply the y-shit
car_dat[sdate:edate][p_col].subtract(car_dat[sdate:edate][p_col].mean()-y_shift).plot(ylabel='nT', xlabel='Time - UT',
                                                            figsize=[6,10])
plt.title(sdate[0:11]+' ULF Wave')

## Analysis

Python offers the widest range of analysis tools for your research and has a huge community to help address problems.

- Basic Anlaysis
  - [Numpy](https://numpy.org/)
  - [SciPy](https://scipy.org/)
- More complex and Machine Learning
  - [Scikit-learn](https://scikit-learn.org/)
  - [Tensor Flow](https://www.tensorflow.org/)
  - [PyTorch](https://pytorch.org/)
  - [Keras](https://keras.io/)
- Heliophysics
  - [PyHC](https://heliopython.org/)
  - [Astropy](https://www.astropy.org/index.html)
  - [Sunpy](https://sunpy.org/)
  - [SpacePy](https://spacepy.github.io/index.html)
  - [pySPEDAS](https://pyspedas.readthedocs.io/en/latest/getting_started.html)
  - [plasmapy](https://docs.plasmapy.org/en/stable/)
  - [pysat](https://pysat.readthedocs.io/en/latest/introduction.html)
  - [AACGM](https://aacgmv2.readthedocs.io/en/latest/)
  - [CDFlib](https://github.com/MAVENSDC/cdflib)
  - [geopack](https://github.com/tsssss/geopack/blob/master/README.md)
  - [IGRF](https://github.com/space-physics/igrf)
  - [PyDarn](https://superdarn.ca/pydarn)

### Filtering

Let try some simple filtering using SciPy (from [Scipy-cookbook](https://scipy-cookbook.readthedocs.io/index.html) and [stackoverflow](https://stackoverflow.com/questions/12093594/how-to-implement-band-pass-butterworth-filter-with-scipy-signal-butter))

In [ ]:
# import scipy functions
from scipy.signal import butter, sosfilt

# define a bandpass butterwork filter
# low and high cutoff frequencies
# fs sampling rate (same units as )
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], btype='band', output='sos')
    return sos

# define and filter the data
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y

In [ ]:
# low and high perios in seconds
# estimated from the figure above
low_period = 250.
high_period = 500.

# low and high frequency cutoffs
lowcut = 1./high_period
highcut = 1./low_period

# sampling frequency
# 1/(Time Resolution)
fs = 1./car_meta[car_meta['code']=='GILL']['Time Resolution']

# Filter the data
f_dat = butter_bandpass_filter(car_dat['GILL_H'], lowcut, highcut, fs,order=5)

# Add the filtered data to our DataFrame
car_dat['GILL_HF'] = f_dat

# Plot the original and filtered data
car_dat[sdate:edate].plot(y=['GILL_H','GILL_HF'],subplots=True,ylabel='nT')



### Exercise 3

Use SciPy to identify the dominant frequency in the ULF wave plotted above. **Hint** use the SciPy periodogram function.

Does the frequency content change as you look at different portions of the day (e.g., if use only data between 13:25-14:25 UT or the entire day)?

### Solution

In [ ]:
from scipy.signal import periodogram
fs = 1.
fp, p_spec = periodogram(car_dat[sdate:edate]['GILL_H'], fs, scaling='density', detrend='constant', window='hann')

plt.loglog(fp*1000,p_spec)

print(f'Maximum Power - {p_spec.max()}')
print(f'Frequency (mHz) of maximum power - {fp[p_spec.argmax()]*1000.}')
print(f'Period (s) of maximum power - {1/fp[p_spec.argmax()]}')